# K-Nearest Neighbors Classifier

K-Nearest Neighbors (KNN) is a simple "model-free" approach to classification and regression. For classification, it's successful where the decision boundary between classes is not smooth. "Fitting" the model involves no work at all, as KNN uses the training data to make predictions. For predicting label classes, the model works by 
1. Calculating the (usually Euclidian) distance of the target observation's feature space relative to each observation in the training dataset.
2. Determine the labels of the K closest observations
3. Choose the mode of those K observations as our predicted label.

For regression, we use the label mean instead of the model. Because prediction is done using the training data, it scales linearly with both the number of training observations and the number of dimensions. Prediction, however, scales with the number of training observations and dimensions. It's also sub-optimal in high dimension feature spaces beause all training vectors are almost equidistant (assuming a Euclidian distance function) from the test vector. Thus, plain KNN is best for situations with smaller, low-dimensionality datasets.

My KNNClassifier is inspired by the KNeighborsClassifier in scikit-learn. I'll build mine using NumPy arrays and the scipy.stats.mode function. For simplicity, I'll limit the scope of the model to classification, and I'll use only Euclidian distance as my distance calculation function. I'll evaluate model using standard functions from sklearn. 

In [1]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from scipy import stats

My *fit* method is to just read the training set's feature vectors and labels into instance variables. My *predict* method uses this algorithm to predict new data:
    1. Populate y_pred with empty values.
    2. For each obs (row) in X_pred,
        a. Determine the distances of the obs from all of the observations in self.X
        b. Sort all of the distances
        c. Subset the first k distances (i.e. the lowest k distances)
        d. Find the mode of those k distances
        e. Append the mode to y_pred

In [2]:
class KNNClassifier:
    def __init__(self, k=1):
        self.k = k

    def euclid_distance(self, target, other):
        '''
        Calculates the Euclidian distance of the target (1xd) array against the other array. 
        "other" could be a single 1xd array, or it could be an nxd matrix.
        '''
        
        return np.linalg.norm(target - other, axis=1)

    def fit(self, X, y):
        '''
        "Fits" the model. Since this is a lazy evaluator, this will just entail reading the 
        training data into self.X & y, so that we can use them in the predict function.
        '''
        
        self.X = X
        self.y = y

    def predict(self, X_pred):
        '''
        Predicts the outcome variable y from inputs X_pred. X_pred is a nxd array where n 
        is the number of observations and d is the number of features. returns y_pred, an 
        nx1 array of predicted outcomes.            
        '''
        
        n_obs = X_pred.shape[0]
        y_pred = np.empty(n_obs)

        for i in range(n_obs):
            distances = self.euclid_distance(X_pred[i], self.X)
            y_sorted  = self.y[distances.argsort()]
            neighbors = y_sorted[:self.k]
            y_pred[i] = stats.mode(neighbors)[0][0]

        return y_pred

Let's load the iris dataset, and split it into randomized train and test data 80/20.

In [3]:
X, y = load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

Now let's create a KNNClassifier object with k=5, and fit it to the training data.

In [4]:
knn = KNNClassifier(k=5)
knn.fit(X_train,y_train)

Then we'll evaluate the model with the test data. I'll calculate the accuracy score and confusion matrix.

In [5]:
y_pred = knn.predict(X_test)
print('Actual & Predicted y values')
print(np.vstack([y_test, y_pred]).T)
print('\n')

knn_accuracy_score = accuracy_score(y_test, y_pred)
print('Accuracy score: {}'.format(knn_accuracy_score))
print('\n')

print('Confusion Matrix')
print(confusion_matrix(y_test, y_pred))

Actual & Predicted y values
[[ 1.  1.]
 [ 0.  0.]
 [ 2.  2.]
 [ 0.  0.]
 [ 1.  1.]
 [ 2.  2.]
 [ 0.  0.]
 [ 1.  1.]
 [ 0.  0.]
 [ 0.  0.]
 [ 1.  1.]
 [ 0.  0.]
 [ 0.  0.]
 [ 1.  1.]
 [ 1.  1.]
 [ 1.  1.]
 [ 0.  0.]
 [ 2.  2.]
 [ 2.  2.]
 [ 0.  0.]
 [ 1.  1.]
 [ 1.  1.]
 [ 2.  2.]
 [ 2.  2.]
 [ 0.  0.]
 [ 1.  1.]
 [ 2.  2.]
 [ 2.  2.]
 [ 0.  0.]
 [ 2.  2.]]


Accuracy score: 1.0


Confusion Matrix
[[11  0  0]
 [ 0 10  0]
 [ 0  0  9]]


With this particular train/test split, it's 100% accurate.

Let's also verify that my results are similar to sklearn's KNN model:

In [8]:
sklearn_knn = KNeighborsClassifier(n_neighbors=5)
sklearn_knn.fit(X_train, y_train)
sklearn_y_pred = sklearn_knn.predict(X_test)

print('Actual & Predicted y values')
print(np.vstack([y_test, sklearn_y_pred]).T)
print('\n')

sklearn_knn_accuracy_score = accuracy_score(y_test, sklearn_y_pred)
print('Accuracy score: {}'.format(sklearn_knn_accuracy_score))
print('\n')

print('Confusion Matrix')
print(confusion_matrix(y_test, sklearn_y_pred))

Actual & Predicted y values
[[1 1]
 [0 0]
 [2 2]
 [0 0]
 [1 1]
 [2 2]
 [0 0]
 [1 1]
 [0 0]
 [0 0]
 [1 1]
 [0 0]
 [0 0]
 [1 1]
 [1 1]
 [1 1]
 [0 0]
 [2 2]
 [2 2]
 [0 0]
 [1 1]
 [1 1]
 [2 2]
 [2 2]
 [0 0]
 [1 1]
 [2 2]
 [2 2]
 [0 0]
 [2 2]]


Accuracy score: 1.0


Confusion Matrix
[[11  0  0]
 [ 0 10  0]
 [ 0  0  9]]


Great.